In [1]:
from owlready2 import *
onto = get_ontology("/home/rudresh/Documents/MSC3/S5/Code/Data/data.owl").load()

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [24]:
# onto.search(iri = "AdaBoost")
temp = 'http://www.semanticweb.org/rudresh/ontologies/2020/1/dataScience-ontology-8#Deep_Learning'
type(temp)
t =IRIS[temp].classes()
print(t)

# onto.search_one(label = "AdaBoost")

# onto.search(iri = "*Boost")

AttributeError: 'classes' property is not defined.

In [3]:
list(onto.classes())

[data.AdaBoost,
 data.Ensemble,
 data.Agnostic_Specifications,
 data.Production,
 data.Apriori_algorithm,
 data.Rule_System,
 data.Arithmetic_Mean,
 data.Statistics,
 data.BI_and_Datawarehousing,
 data.Coreskills,
 data.Back-Propagation,
 data.Neural_Networks,
 data.Bayesian,
 data.Learning_Algorithm,
 data.Boosting,
 data.BootStrap_Sampling,
 data.Model_Validation,
 data.Business_Analysis_and_IT_Projects,
 data.C4.5,
 data.Decision_Tree,
 data.C5.0,
 data.Cloud,
 data.Cluster_Validity,
 data.Clustering,
 data.Coefficient_of_variation,
 data.Conditional_Decision_Trees,
 data.Confusion_Matrix,
 data.DomainConcept,
 data.Cross_Validation,
 data.Data_Cleaning,
 data.Data_Science,
 data.Data_Engineering,
 data.Data_Prepration,
 data.Data_Visualization,
 data.Databases,
 data.Dataflow,
 data.Pradigms,
 data.Decision_Stump,
 data.Deep_Learning,
 data.Deep_belief_network,
 data.Dimensionality_Reduction,
 data.Distributed_ML_Libraries,
 data.Eclat_algorithm,
 data.Elastic_Net,
 data.Regulariza

In [4]:
import spacy
from nltk import RegexpParser, tree
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import itertools

In [5]:
GRAMMAR = "DBW_CONCEPT: {<JJ.*>*<HYPH>*<JJ.*>*<HYPH>*<NN.*>*<HYPH>*<NN.*>+}" #good for syntactic

In [6]:
paper = {
        "abstract": "AI talent acquisition platform: Designed, created and deployed a framework using Conditional random field and Stanford core NLP model to parse the resume and job positions for a military organisation NATO.◦ IBM Cloud: Created and deployed docker and cloud foundry application on IBM Bluemix. Graph QL: Developed a GraphQL API to respond for the queries which predicts the estimated salary from the model based on location ,industry and job title inputs. Graph Database: Designed, deployed a Neo4j graph database script to migrate data from MySQL To Neo4j. Web Crawler: Created a python crawler script to scrape the data from the HTML pages using beautiful soup. ◦ Text Analytic: Created a rule based model to work as an extractor that pulls structured information from unstructured or semi-structured text using Annotated Query Language.Data Analyst: Used different statistical technique to gather data insights and presented in depth findings to improve business strategies and operations. ◦ Oracle Database Administration: Scheduled the database replication script to sync the data. Performed the database refresh, patch and upgrade on Oracle 11gR1/11gR2. Analyzed the database performance. Created partition and indexes on tables to improve the performance. Fine-tuned SQL queries to improve the efficiency. Data Migration: Created the PL/SQL stored procedures,functions and packages to migrate data from oracle db to Maria db using oracle JDBC. Unix Shell Scripting: Created a unix shell script to automate the daily activity task as per client requirement and deployed it on Cron server. Team Management: Managed a team of 4 people by assigning, guiding the task and making deliveries to client. Sentiment Analysis: Sentiment analysis on the amazon reviews using 1D convolution neural network. • Ship Detection in Satellite imagery: Ship detection in optical satellite images using Resnet-Unet model. • Fake reviews detection: Fake review detection on online platform based on user behaviors and text reviews.• Job post and resume matching : Applying the concept of NLP, ML and Multiple criteria sorting method to match and score the resume with job posting. it also involves scraping job post, building taxonomy and ontology.",
        "keywords": "data mining, computer science"
        }

In [7]:
class Paper:
    """ A simple abstraction layer for working on the paper object"""
    
    def __init__(self, paper = None, modules = None):
        """ Initialising the ontology class
        """
        self.title = None
        self.abstract = None
        self.keywords = None
        self._text = None
        self.chunks = None
        self.text_attr = ('title', 'abstract', 'keywords')
        self.tagger = spacy.load('en_core_web_sm', disable=['ner'])
        
        if modules is not None:
            self.modules = modules
        
        if paper is not None:
            self.set_paper(paper)
            
    
    
    def set_paper(self, paper):
        """Function that initializes the paper variable in the class.
        Args:
            paper (either string or dictionary): The paper to analyse. It can be a full string in which the content
            is already merged or a dictionary  {"title": "","abstract": "","keywords": ""}.
        """
        self.title = None
        self.abstract = None
        self.keywords = None
        self._text = None
        self.semantic_chunks = None
        self.syntactic_chunks = None
        
        try:
            if isinstance(paper, dict):              
                for attr in self.text_attr:
                    try: 
                        setattr(self, attr, paper[attr])
                    except KeyError:
                        continue
                    
                self.treat_keywords()
                self.text()
                
            elif isinstance(paper, str):
                self._text = paper.strip()
            
            else:
                raise TypeError("Error: Unrecognised paper format")
                return
        
            self.pre_process()
            
        except TypeError:
            pass


    def text(self):
        """ Text aggregator
        """
        attr_text = [getattr(self, attr) for attr in self.text_attr]
        self._text = '. '.join((s.rstrip('.') for s in attr_text if s is not None))


    def treat_keywords(self):
        """ Function that handles different version of keyword field
        """
        if self.keywords is None:
            return
        if isinstance(self.keywords, list):
            self.keywords = ', '.join(self.keywords)


    def part_of_speech_tagger(self, doc):
        """ Part of speech tagger
        Returns:
            text (string): single token
            tag_ (string): POS tag
        """
        for token in doc:
            if token.tag_:
                yield token.text, token.tag_
                
    def remove_root_verb(self, doc):
        """ Creates a string in which it removes verbs that are also root of the tree
        """   
        new_document = doc.text        
        items_to_remove = [(token.text,token.dep_,token.pos_,token.idx, token.idx+len(token.text)) for token in doc if (token.pos_ == "VERB" and token.dep_ == "ROOT")]
        for item in reversed(items_to_remove):
            new_document = new_document[:item[3]] + "." + new_document[item[4]:]
        return new_document
            

    def extraxt_semantic_chuncks(self, pos_tags):
        """ Extract chunks of text from the paper taking advantage of the parts of speech previously extracted.
        It uses a grammar
        Returns:
            chunks (list): list of all chunks of text 
        """
        grammar_parser = RegexpParser(GRAMMAR)
        chunks = list()
        pos_tags_with_grammar = grammar_parser.parse(pos_tags)
        #print(pos_tags_with_grammar)
        for node in pos_tags_with_grammar:
            if isinstance(node, tree.Tree) and node.label() == 'DBW_CONCEPT': # if matches our grammar 
                chunk = ''
                for leaf in node.leaves():
                    concept_chunk = leaf[0]
                    concept_chunk = re.sub('[\=\,\…\’\'\+\-\–\“\”\"\/\‘\[\]\®\™\%]', ' ', concept_chunk)
                    concept_chunk = re.sub('\.$|^\.', '', concept_chunk)
                    concept_chunk = concept_chunk.lower().strip()
                    chunk += ' ' + concept_chunk
                chunk = re.sub('\.+', '.', chunk)
                chunk = re.sub('\s+', ' ', chunk)
                chunks.append(chunk)
        return chunks
    
    
    def extraxt_syntactic_chuncks(self, document):
        """ Extract chunks of text from the paper, using stopwords as delimiter.
        It uses a grammar
        Returns:
            chunks (list): list of all chunks of text 
        """
        tokenizer = RegexpTokenizer(r'[\w\-\(\)]*')
        tokens = tokenizer.tokenize(document)
        filtered_words = [a for a in [w if w not in stopwords.words('english') else ':delimiter:' for w in tokens] if a != '']
        matrix_of_tokens = [list(g) for k,g in itertools.groupby(filtered_words,lambda x: x == ':delimiter:') if not k]
        return [" ".join(row).lower() for row in matrix_of_tokens]
    
                
    def pre_process(self):
        """ Pre-processes the paper: identifies the parts of speech and then extracts chunks using a grammar
        """
        ##################### Tagger with spaCy.io
        doc = self.tagger(self._text)
        
        # =============================================================================
        #         SYNTACTIC
        # =============================================================================
        if self.modules == 'syntactic' or self.modules == 'both':
            ##################### Getting new filtered document removing ROOT NODES(that are VERBS)
            new_filtered_document = self.remove_root_verb(doc)
            ##################### Extraxting chunks of text base on stop-words
            self.syntactic_chunks = self.extraxt_syntactic_chuncks(new_filtered_document)
        # =============================================================================
        #         SEMANTIC
        # =============================================================================
        if self.modules == 'semantic' or self.modules == 'both':
            ##################### Getting text and POS in the right configuration
            pos_tags = self.part_of_speech_tagger(doc)
            ##################### Applying grammar          
            self.semantic_chunks = self.extraxt_semantic_chuncks(list(pos_tags))  
     
        

    
    def get_text(self):
        """Returns the text of the paper
        """
        return self._text
    
    def get_new_text(self):
        """Returns the text of the paper
        """
        return self.new_text
    
    def get_semantic_chunks(self):
        """Returns the chunks extracted from the paper (used by the semantic module)
        """
        return self.semantic_chunks
    
    def get_syntactic_chunks(self):
        """Returns the chunks extracted from the paper (used by the syntactic module)
        """
        return self.syntactic_chunks
    
    def set_modules(self, modules):
        """Setter for the modules variable"""
        self.modules = modules

In [8]:
smallPaper = Paper(paper, 'semantic')

smallPaper.set_paper(paper)
smallPaper.pre_process()
print(smallPaper.get_semantic_chunks())

[' talent acquisition platform', ' framework', ' conditional random field', ' stanford core nlp model', ' resume', ' job positions', ' military organisation nato', ' ◦ ibm cloud', ' docker', ' foundry application', ' ibm bluemix', ' graph ql', ' graphql api', ' queries', ' salary', ' model', ' location', ' industry', ' job title inputs', ' graph database', ' neo4j graph database script', ' data', ' mysql', ' web crawler', ' python crawler script', ' data', ' html pages', ' beautiful soup', ' ◦ text analytic', ' rule', ' model', ' extractor', ' structured information', ' semi structured text', ' annotated query language', ' data analyst', ' different statistical technique', ' data insights', ' depth findings', ' business strategies', ' operations', ' ◦ oracle database administration', ' database replication script', ' data', ' database refresh', ' patch', ' oracle', ' database performance', ' partition', ' indexes', ' tables', ' performance', ' sql', ' efficiency', ' data migration', ' 

In [9]:
from nltk import everygrams
from kneed import KneeLocator

import warnings

class Semantic:
    """ A simple abstraction layer for using the Semantic module of the CSO classifier """
    
    def __init__(self, model = None, cso = None, paper = None):
        """Function that initialises an object of class CSOClassifierSemantic and all its members.
        Args:
            model (dictionary): word2vec model.
            cso (dictionary): Computer Science Ontology
            paper (dictionary): paper{"title":"...","abstract":"...","keywords":"..."} the paper.
        """
        self.cso = cso                  #Stores the CSO Ontology
        self.paper = paper              #Paper to analyse
        self.model = model              #contains the cached model          
        self.min_similarity = 0.94      #Initialises the min_similarity
        self.explanation = dict()
        
        
    def set_paper(self, paper):
        """Function that initializes the paper variable in the class.
        Args:
            paper (either string or dictionary): The paper to analyse. It can be a full string in which the content
            is already merged or a dictionary  {"title": "","abstract": "","keywords": ""}.
        """
        self.paper = paper
        self.reset_explanation()
        
    
    def set_min_similarity(self, min_similarity):
        """Function that initializes the minimum similarity variable.
        Args:
            min_similarity (float): value of min_similarity between 0 and 1.
        """
        self.min_similarity = min_similarity
        
        
    def reset_explanation(self):
        """ Resetting the explanation 
        """
        self.explanation = dict()
        
        
    def get_explanation(self):
        """ Returns the explanation 
        """
        return self.explanation
 
    
    def classify_semantic(self):
        """Function that classifies the paper on a semantic level. This semantic module follows four steps: 
            (i) entity extraction, 
            (ii) CSO concept identification, 
            (iii) concept ranking, and 
            (iv) concept selection.
        Args:
            processed_embeddings (dictionary): This dictionary saves the matches between word embeddings and terms in CSO. It is useful when processing in batch mode.
        Returns:
            final_topics (list): list of identified topics.
        """     

        ##################### Core analysis
        found_topics, explanation = self.find_topics(self.paper.get_semantic_chunks())
    
        ##################### Ranking
        final_topics = self.rank_topics(found_topics, explanation) 
             
        return final_topics


    def find_topics(self, concepts):
        """Function that identifies topics starting from the ngram forund in the paper
        Args:
            concepts (list): Chuncks of text to analyse.
        Returns:
            found_topics (dict): cdictionary containing the identified topics.
        """
        
        # Set up
        found_topics = dict() # to store the matched topics
        explanation = dict()

        # finding matches
        for concept in concepts:
            evgrams = everygrams(concept.split(), 1, 3) # list of unigrams, bigrams, trigrams
            for grams in evgrams:
                gram = "_".join(grams)
                gram_without_underscore = " ".join(grams)
                #### Finding similar words contained in the model
                
                list_of_matched_topics = []

                if self.model.check_word_in_model(gram):
                    list_of_matched_topics = self.model.get_words_from_model(gram)
                    
                else:                    
                    list_of_matched_topics = self.match_ngram(grams)
                                    
                            
                for topic_item in list_of_matched_topics:
                        
                    topic = topic_item["topic"]
                    m     = topic_item["sim_t"]
                    wet   = topic_item["wet"]
                    sim   = topic_item["sim_w"]
                    
                    
                    if m >= self.min_similarity and topic in self.cso.topics_wu:
                        
    
                        if topic in found_topics:
                            #tracking this match
                            found_topics[topic]["times"] += 1
                            
                            found_topics[topic]["gram_similarity"].append(sim)
    
                            #tracking the matched gram
                            if gram in found_topics[topic]["grams"]: 
                                found_topics[topic]["grams"][gram] += 1
                            else:
                                found_topics[topic]["grams"][gram] = 1
    
                            #tracking the most similar gram to the topic
                            if m > found_topics[topic]["embedding_similarity"]:
                                found_topics[topic]["embedding_similarity"] = m 
                                found_topics[topic]["embedding_matched"] = wet
    
                        else:
                            #creating new topic in the result set
                            found_topics[topic] = {'grams': {gram:1},
                                                    'embedding_matched': wet,
                                                    'embedding_similarity': m,
                                                    'gram_similarity':[sim],
                                                    'times': 1, 
                                                    'topic':topic}

                        
                        
                        if sim == 1:
                            found_topics[topic]["syntactic"] = True
    
                            
                            
                        primary_label_topic = self.cso.get_primary_label_wu(topic)
                        if primary_label_topic not in explanation:
                            explanation[primary_label_topic] = set()
                        
                        explanation[primary_label_topic].add(gram_without_underscore)
        
        return found_topics, explanation
    
    
    def match_ngram(self, grams, merge=True):
        """
        Args:
            grams (list): list of tokens to be analysed and founf in the model
            merge (boolean): #Allows to combine the topics of mutiple tokens, when analysing 2-grams or 3-grams
        
        Returns:
            list_of_matched_topics (list): containing of all found topics 
        """
        
        list_of_matched_topics = list()
        if len(grams) > 1 and merge:
            
            temp_list_of_matches = {}
            
            list_of_merged_topics = {}
            
            for gram in grams:
                if self.model.check_word_in_model(gram):
                    list_of_matched_topics_t = self.model.get_words_from_model(gram)
                    for topic_item in list_of_matched_topics_t:
                        temp_list_of_matches[topic_item["topic"]] = topic_item
                        try:
                            list_of_merged_topics[topic_item["topic"]] += 1
                        except KeyError:
                            list_of_merged_topics[topic_item["topic"]] = 1
                        
            for topic_x, value in list_of_merged_topics.items():
                if value >= len(grams):
                    list_of_matched_topics.append(temp_list_of_matches[topic_x])
        
        return list_of_matched_topics
                    
        
    def rank_topics(self, found_topics, explanation):
        """ Function that ranks the list of found topics. It also cleans the explanation accordingly
        
        Args:
            found_topics (dictionary): contains all information about the found topics
            explanation (dictionary): contains information about the explanation of topics
        
        Returns:
            final_topics (list): list of final topics
        """
        max_value = 0
        scores = []
        for tp,topic in found_topics.items(): 
            topic["score"] = topic["times"] * len(topic['grams'].keys())
            scores.append(topic["score"])
            if topic["score"] > max_value:
                max_value = topic["score"]

        for tp,topic in found_topics.items():            
            if "syntactic" in topic:
                topic["score"] = max_value
                
                
             
            
        # Selection of unique topics  
        unique_topics = {}
        for tp,topic in found_topics.items():
            prim_label = self.cso.get_primary_label_wu(tp)
            if prim_label in unique_topics:
                if unique_topics[prim_label] < topic["score"]:
                    unique_topics[prim_label] = topic["score"]
            else:
                unique_topics[prim_label] = topic["score"]
        
        # ranking topics by their score. High-scored topics go on top
        sort_t = sorted(unique_topics.items(), key=lambda v: v[1], reverse=True)
        #sort_t = sorted(found_topics.items(), key=lambda k: k[1]['score'], reverse=True)
        
        
        # perform 
        vals = []
        for tp in sort_t:
            vals.append(tp[1]) #in 0, there is the topic, in 1 there is the info
        
        
        #### suppressing some warnings that can be raised by the kneed library
        warnings.filterwarnings("ignore")
        try:
            x = range(1,len(vals)+1) 
            kn = KneeLocator(x, vals, direction='decreasing')
            if kn.knee is None:
                #print("I performed a different identification of knee")
                kn = KneeLocator(x, vals, curve='convex', direction='decreasing')
        except ValueError:
            pass
        
        ##################### Pruning
        
        try: 
            knee = int(kn.knee)
        except TypeError:
            knee = 0
        except UnboundLocalError:
            knee = 0
            
        if knee > 5:
            try:
                knee += 0
            except TypeError:
                print("ERROR: ",kn.knee," ",knee, " ", len(sort_t))
            
        else:
            try:
                if sort_t[0][1] == sort_t[4][1]:
                    top = sort_t[0][1]
                    test_topics = [item[1] for item in sort_t if item[1]==top] 
                    knee = len(test_topics)
    
                else:
                    knee = 5
            except IndexError:
                knee = len(sort_t)

        final_topics = []
        final_topics = [self.cso.get_topic_wu(sort_t[i][0]) for i in range(0,knee)] 
        self.reset_explanation()
        self.explanation = {self.cso.topics_wu[sort_t[i][0]]: explanation[sort_t[i][0]] for i in range(0,knee)}

        return final_topics

In [ ]:
semantic_topic = Semantic()